# Tabular Data Federated Statistics 

Before we perform machine learning tasks on the Higgs data, let's examine the statistics of the dataset. 


## Setup NVFLARE

Follow [Getting Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to set up a virtual environment and install NVFLARE.

You can also follow this [notebook](https://github.com/NVIDIA/NVFlare/blob/main/examples/nvflare_setup.ipynb) to get set up.

> Make sure you have installed nvflare from **terminal** 




## Install requirements
assuming the current directory is 'higgs/stats'

In [ ]:
! pwd

In [ ]:
%pip install -r requirements.txt


## Prepare data
Please reference [prepare_higgs_data](../prepare_data.ipynb) notebooks. Pay attention to the current location. You need to switch "higgs" directory to run the data split.
    

Now we have our data prepared, let's first take a look at these data.

In [ ]:
features = ["label", "lepton_pt", "lepton_eta", "lepton_phi", "missing_energy_magnitude", "missing_energy_phi", "jet_1_pt", "jet_1_eta", "jet_1_phi", "jet_1_b_tag", "jet_2_pt", "jet_2_eta", "jet_2_phi", "jet_2_b_tag", "jet_3_pt", "jet_3_eta", "jet_3_phi", "jet_3_b_tag",\
            "jet_4_pt", "jet_4_eta", "jet_4_phi", "jet_4_b_tag", \
            "m_jj", "m_jjj", "m_lv", "m_jlv", "m_bb", "m_wbb", "m_wwbb"]

In [ ]:
features

In [ ]:
import numpy as np
import pandas as pd

df: pd.DataFrame = pd.read_csv("/tmp/nvflare/dataset/output/site-1.csv", names=features, sep=r"\s*,\s*", engine="python", na_values="?")

In [ ]:
df

## Create a statistics calculator for the local tabular dataset


```
import csv
from typing import Dict, List, Optional

import numpy as np
import pandas as pd
from pandas.core.series import Series

from nvflare.apis.fl_context import FLContext
from nvflare.app_common.abstract.statistics_spec import BinRange, Feature, Histogram, HistogramType, Statistics
from nvflare.app_common.statistics.numpy_utils import dtype_to_data_type, get_std_histogram_buckets


class DFStatistics(Statistics):
    def __init__(self, data_root_dir: str):
        super().__init__()
        self.data_root_dir = data_root_dir
        self.data: Optional[Dict[str, pd.DataFrame]] = None
        self.data_features = None

    def load_features(self, fl_ctx: FLContext) -> List:
        client_name = self.get_client_name(fl_ctx)
        try:
            data_path = f"{self.data_root_dir}/{client_name}_header.csv"

            features = []
            with open(data_path, 'r') as file:
                # Create a CSV reader object
                csv_reader = csv.reader(file)
                line_list = next(csv_reader)
                features = line_list
            return features
        except Exception as e:
            raise Exception(f"Load header for client {client_name} failed! {e}")

    def load_data(self, fl_ctx: FLContext) -> Dict[str, pd.DataFrame]:
        client_name = self.get_client_name(fl_ctx)
        try:
            data_path = f"{self.data_root_dir}/{client_name}.csv"
            # example of load data from CSV
            df: pd.DataFrame = pd.read_csv(
                data_path, names=self.data_features, sep=r"\s*,\s*", engine="python", na_values="?"
            )
            return {"train": df}

        except Exception as e:
            raise Exception(f"Load data for client {client_name} failed! {e}")

    def get_client_name(self, fl_ctx):
        client_name = fl_ctx.get_identity_name() if fl_ctx is not None else "site-1"
        if fl_ctx:
            self.log_info(fl_ctx, f"load data for client {client_name}")
        else:
            print(f"load data for client {client_name}")
        return client_name

    def initialize(self, fl_ctx: FLContext):
        self.data_features = self.load_features(fl_ctx)
        self.data = self.load_data(fl_ctx)
        if self.data is None:
            raise ValueError("data is not loaded. make sure the data is loaded")

    def features(self) -> Dict[str, List[Feature]]:
        results: Dict[str, List[Feature]] = {}
        for ds_name in self.data:
            df = self.data[ds_name]
            results[ds_name] = []
            for feature_name in df:
                data_type = dtype_to_data_type(df[feature_name].dtype)
                results[ds_name].append(Feature(feature_name, data_type))

        return results

    def count(self, dataset_name: str, feature_name: str) -> int:
        df: pd.DataFrame = self.data[dataset_name]
        return df[feature_name].count()

    def sum(self, dataset_name: str, feature_name: str) -> float:
        df: pd.DataFrame = self.data[dataset_name]
        return df[feature_name].sum().item()

    def mean(self, dataset_name: str, feature_name: str) -> float:

        count: int = self.count(dataset_name, feature_name)
        sum_value: float = self.sum(dataset_name, feature_name)
        return sum_value / count

    def stddev(self, dataset_name: str, feature_name: str) -> float:
        df = self.data[dataset_name]
        return df[feature_name].std().item()

    def variance_with_mean(
        self, dataset_name: str, feature_name: str, global_mean: float, global_count: float
    ) -> float:
        df = self.data[dataset_name]
        tmp = (df[feature_name] - global_mean) * (df[feature_name] - global_mean)
        variance = tmp.sum() / (global_count - 1)
        return variance.item()

    def histogram(
        self, dataset_name: str, feature_name: str, num_of_bins: int, global_min_value: float, global_max_value: float
    ) -> Histogram:

        num_of_bins: int = num_of_bins

        df = self.data[dataset_name]
        feature: Series = df[feature_name]
        flattened = feature.ravel()
        flattened = flattened[flattened != np.array(None)]
        buckets = get_std_histogram_buckets(flattened, num_of_bins, BinRange(global_min_value, global_max_value))
        return Histogram(HistogramType.STANDARD, buckets)

    def max_value(self, dataset_name: str, feature_name: str) -> float:
        """this is needed for histogram calculation, not used for reporting"""

        df = self.data[dataset_name]
        return df[feature_name].max()

    def min_value(self, dataset_name: str, feature_name: str) -> float:
        """this is needed for histogram calculation, not used for reporting"""

        df = self.data[dataset_name]
        return df[feature_name].min()


```

Let's see if the code works. 

In [ ]:
cd code

In [ ]:
from df_stats import DFStatistics

df_stats_cal = DFStatistics(data_root_dir = "/tmp/nvflare/dataset/output")

# We use fl_ctx = None for local calculation ( where the data set default to "site-1.csv", so we can explore the stats locally without federated settings. 
df_stats_cal.initialize(fl_ctx = None)


In [ ]:
data_features = df_stats_cal.features()

In [ ]:
data_features

In [ ]:
df_stats_cal.count("train", "lepton_pt")

In [ ]:
df_stats_cal.mean("train", "lepton_pt")

In [ ]:
df_stats_cal.mean("train", "m_wwbb")

In [ ]:
df_stats_cal.stddev("train", "m_wwbb")

In [ ]:
df_stats_cal.histogram("train", "lepton_pt", 20, 0, 10)

Great ! The code works. Let's move to the federated statistics calculations. Befor we do that, we need to move back to the parent directory of code

In [ ]:
cd ../.

## Create Federated Statistics Job

We are going to use NVFLARE job cli to create job. For detailed instructions on Job CLI, please follow the [job cli tutorial](https://github.com/NVIDIA/NVFlare/blob/main/examples/tutorials/job_cli.ipynb)

Let's check the available job templates, we are going to use one of the existing job templates and modify it to fit our needs. The job template is nothing but server and client-side job configurations.

In [ ]:
! nvflare job list_templates

there is "stats_df" job template, which what we need. We are going to use that. Now, use ```nvflare job create``` command
We would like to use our new df_statistics.py file we just tested

In [ ]:
! nvflare job create -w stats_df -j /tmp/nvflare/jobs/stats_df -sd code -force

In [ ]:
! tree /tmp/nvflare/jobs/stats_df  

Let's modify the server configuration to set the bin = 20, global min_max range in [0,10] instead of [0,120] and stats_writer output path  "statistics/adults_stats.json"

In [ ]:
! nvflare job create -w stats_df -force -j /tmp/nvflare/jobs/stats_df -sd code -f config_fed_server.conf bins=20 range="[0,10]" output_path="statistics/stats.json"

In [ ]:
!cat /tmp/nvflare/jobs/stats_df/app/config/config_fed_server.conf         

Now, look at the client configuration, we notice that the job template component configuration 
```
components = [
  {
    id = "df_stats_generator"
    path = "df_statistics.DFStatistics"
    args {
      data_path = "data.csv"
    }
  }
```

is different from our new DFStatistics class, where the arguments are
features, data_root_dir not "data_path". So we will need to modify that. 

```

class DFStatistics(Statistics):
    def __init__(self, data_root_dir: str):
        super().__init__()
        self.data_root_dir = data_root_dir
        self.data: Optional[Dict[str, pd.DataFrame]] = None
        self.data_features = None
```


In [ ]:
!cat /tmp/nvflare/jobs/stats_df/app/config/config_fed_client.conf 

what we need to do are the followings
1. remove data_path argument
2. add data_root_dir arguments
3. change the path of the DFStatistics class from 'df_statistics.DFStatistics' to df_stats.DFStatistics'

We use the following syntax to do this (you can always open it with your editing tool to direct edit the file). 



In [ ]:
! nvflare job create -w stats_df -force -j /tmp/nvflare/jobs/stats_df \-sd code \
-f config_fed_server.conf \
   bins=20 \
   range="[0,10]" \
   output_path="statistics/stats.json" \
-f config_fed_client.conf \
   components[0].path="df_stats.DFStatistics" \
   components[0].args.data_path- \
   components[0].args.data_root_dir="/tmp/nvflare/dataset/output" -debug

   

In [ ]:
!tree  /tmp/nvflare/jobs/stats_df  



## Run job in FL Simulator

Now we can run the job with simulator. 

```
nvflare simulator  /tmp/nvflare/jobs/stats_df  -w /tmp/nvflare/tabular/stats_df  -n 3 -t 3
```

In [ ]:
!nvflare simulator  /tmp/nvflare/jobs/stats_df  -w /tmp/nvflare/tabular/stats_df  -n 3 -t 3



The results are stored in 
```
/tmp/nvflare/tabular/stats_df/simulate_job/statistics/stats.json
```


In [ ]:
!ls -al  /tmp/nvflare/tabular/stats_df/simulate_job/statistics/


## Visualization


In [ ]:
! cp /tmp/nvflare/tabular/stats_df/simulate_job/statistics//stats.json ./.

In [ ]:

import json
import pandas as pd
from nvflare.app_opt.statistics.visualization.statistics_visualization import Visualization
with open('stats.json', 'r') as f:
    data = json.load(f)

vis = Visualization()
vis.show_stats(data = data)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100%  depth:100% !important; }</style>"))

In [ ]:
vis.show_histograms(data = data, plot_type="main")

The global and local histograms differences are none as we are using the same dataset for all clients. 

## We are done !
Congratulations! you have just completed the federated stats calulation for tabular data. 

If you would like to see a detailed discussion regarding privacy filtering, please checkout the example in [federated statistics](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/federated-statistics) examples





